# Verify Waymo Dataset

In [1]:
!ls waymo_data_v120/training

0-of-10.tfrecord  3-of-10.tfrecord  6-of-10.tfrecord  9-of-10.tfrecord
1-of-10.tfrecord  4-of-10.tfrecord  7-of-10.tfrecord
2-of-10.tfrecord  5-of-10.tfrecord  8-of-10.tfrecord


In [2]:
!ls waymo_data_v120/validation

0-of-10.tfrecord  3-of-10.tfrecord  6-of-10.tfrecord  9-of-10.tfrecord
1-of-10.tfrecord  4-of-10.tfrecord  7-of-10.tfrecord
2-of-10.tfrecord  5-of-10.tfrecord  8-of-10.tfrecord


# Download Model from Tensorflow 2 Model Zoo

In [21]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d4_coco17_tpu-32.tar.gz

--2021-04-02 20:38:06--  http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d4_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 2607:f8b0:4005:805::2010, 216.58.194.208
Connecting to download.tensorflow.org (download.tensorflow.org)|2607:f8b0:4005:805::2010|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 158496448 (151M) [application/x-tar]
Saving to: ‘efficientdet_d4_coco17_tpu-32.tar.gz’

efficientdet_d4_coc 100%[===================>] 151.15M  10.1MB/s    in 15s     

2021-04-02 20:38:21 (10.1 MB/s) - ‘efficientdet_d4_coco17_tpu-32.tar.gz’ saved [158496448/158496448]



In [23]:
!tar -xf efficientdet_d4_coco17_tpu-32.tar.gz

# Configure Model/Parameters

In [31]:
MODELS_CONFIG = {
    'efficientdet_d7': {
        'model_name': 'efficientdet_d7_coco17_tpu-32',
        'base_pipeline_file': 'efficientdet_d7_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d7_coco17_tpu-32.tar.gz',
        'batch_size':7
    },
    'efficientdet_d4': {
        'model_name': 'efficientdet_d4_coco17_tpu-32',
        'base_pipeline_file': 'efficientdet_d4_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d4_coco17_tpu-32.tar.gz',
        'batch_size':14
    }
}

chosen_model = 'efficientdet_d4'
num_steps = 50000
num_eval_steps = 500

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']
batch_size = MODELS_CONFIG[chosen_model]['batch_size']
pipeline_fname = './'+ model_name + '/pipeline.config'
fine_tune_checkpoint = './' + model_name + '/checkpoint/ckpt-0'
label_map_pbtxt_fname = 'waymo_labelmap.txt'
train_record_fname = "./waymo_data_v120/training/*"
test_record_fname = "./waymo_data_v120/validation/*"

In [32]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)
num_classes

3

In [33]:
#write custom configuration file by slotting our dataset, model checkpoint, and training parameters into the base pipeline file

import re
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
        
    f.write(s)


writing custom configuration file


# Training

In [34]:
PIPELINE_CONFIG_PATH='./pipeline_file.config'
MODEL_DIR='./outputmodel_waymo_v120_efficientdet_d4'
!python models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={PIPELINE_CONFIG_PATH} \
    --model_dir={MODEL_DIR} \
    --num_workers=7 \
    --alsologtostderr

2021-04-02 20:56:34.999545: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-02 20:56:36.262192: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-02 20:56:36.262843: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-02 20:56:36.322380: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Tesla V100S-PCIE-32GB computeCapability: 7.0
coreClock: 1.597GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 1.03TiB/s
2021-04-02 20:56:36.324155: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:02:00.0 name: Tesla V100S-PCIE-32GB computeCapability: 7.0
coreClock: 1.597GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 1.03TiB/s
2021-

# Evaluation

In [37]:
PIPELINE_CONFIG_PATH='./pipeline_file.config'
MODEL_DIR='./outputmodel_waymo_v120_efficientdet_d4'
CHECKPOINT_DIR='./outputmodel_waymo_v120_efficientdet_d4'
!python models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={PIPELINE_CONFIG_PATH} \
    --model_dir={MODEL_DIR} \
    --checkpoint_dir={CHECKPOINT_DIR} \
    --num_workers=7 \
    --alsologtostderr

2021-04-03 12:29:39.053737: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/xenowulf/anaconda3/envs/tf/lib/python3.7/site-packages/scipy/__init__.py:140: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.16.0)
  UserWarning)
2021-04-03 12:29:40.319450: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-03 12:29:40.320039: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-03 12:29:40.647452: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Tesla V100S-PCIE-32GB computeCapability: 7.0
coreClock: 1.597GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 1.03TiB/s
2021-04-03 12:29:40.649110: I tensorflow/core/common_runtime/gpu/gpu_device.cc